# Лабораторная работа №6. Линейная регрессия

Выполните следующие задания:

1. Откройте в файл в Google Colab (используйте собственный форк репозитория).
2. Решите задачи.
3. Сохраните результат в ваш репозиторий github в директорию ./les06
4. Создайте pull request в репозиторий https://github.com/chebotarevsa/dap-2024. Название pull request должно иметь формат "<Номер лабораторной работы>  <Номер группы> <ФИО>"
5. Сдайте работу в системе "Пегас", в отчет укажите ссылку на pull request

Набор данных Diabetes (Диабет) содержит 442 образца с 10-ю признаками: возраст, пол, индекс массы тела, средний показатель давления крови и шесть измерений сыворотки крови. Целевое значение - количественный показатель прогрессирования заболевания через год после анализов. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Извлечение данных

In [ ]:
from sklearn import datasets
diabetes = datasets.load_diabetes()

1. Выведете описание набора данных и наименование признаков.

In [ ]:
diabetes = datasets.load_diabetes()

print("=== Описание набора данных ===")
print(diabetes.DESCR)

print("\n=== Названия признаков ===")
print(diabetes.feature_names)

### Подготовка данных

2. Из загруженного набора данных создайте DataFrame, содержащий как признаки, так и целевое значение. Выведите первые 5 строк набора.

In [ ]:
data = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
data['target'] = diabetes.target

print("Первые 5 строк набора данных:")
data.head()

3. Выведете информацию о типах данных в наборе. Имеются ли в наборе категориальные признаки? Имеются ли в наборе данные имеющие значение null?

In [ ]:
print("Информация о наборе данных:")
data.info()

print("\nПроверка на пропущенные значения:")
print(data.isnull().sum())

# Вывод:
# Все признаки — float64, категориальных нет
# Пропущенных значений (null) — нет

## Исследование данных

4. Постройте матрицу корреляции.

In [ ]:
plt.figure(figsize=(12, 10))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Матрица корреляции признаков и целевой переменной', fontsize=16)
plt.tight_layout()
plt.show()

# Находим признак с максимальной корреляцией с target
best_feature = correlation_matrix['target'].drop('target').idxmax()
best_corr = correlation_matrix.loc[best_feature, 'target']
print(f"Признак с наибольшей корреляцией: {best_feature} (r = {best_corr:.3f})")

5. Постройте диаграмму рассеяния целевого значение и признака, коэффициент корреляции которого с  целевым значением, самый высокий.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data[best_feature], data['target'], alpha=0.6, color='#e74c3c')
plt.xlabel(best_feature)
plt.ylabel('Прогрессирование диабета (target)')
plt.title(f'Зависимость прогрессирования диабета от {best_feature}\nКоэффициент корреляции = {best_corr:.3f}')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

6. Сформируйте набор признаков (X) из 5 признаков с самым высоким коэффициентом корреляции с целевым значением. Сформируйте набор для целевого значения (y).

In [ ]:
top5_features = correlation_matrix['target'].drop('target').abs().sort_values(ascending=False).head(5).index.tolist()
print("Топ-5 признаков по корреляции с target:")
for i, feat in enumerate(top5_features, 1):
    corr = correlation_matrix.loc[feat, 'target']
    print(f"{i}. {feat}: {corr:.3f}")

X = data[top5_features]
y = data['target']

## Предсказательная модель

7. Разделите набор данных на два, одни для обучения модели другой для проверки. Тестовый набор должен содержать 25 процентов данных.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape[0]} объектов")
print(f"Размер тестовой выборки: {X_test.shape[0]} объектов")

8. Выполните обучение модели.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

print("Модель обучена!")
print(f"Коэффициенты модели: {np.round(model.coef_, 3)}")
print(f"Свободный член (intercept): {model.intercept_:.3f}")

## Проверка модели

9. Расчитайте Root mean squared error (RMSE)

In [ ]:
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

10. Расчитайте R² (коэффициент детерминации)

In [ ]:
r2 = r2_score(y_test, y_pred)
print(f"R² (коэффициент детерминации): {r2:.4f}")

# Интерпретация:
if r2 > 0.7:
    quality = "отличная"
elif r2 > 0.5:
    quality = "хорошая"
elif r2 > 0.3:
    quality = "удовлетворительная"
else:
    quality = "слабая"

print(f"→ Качество модели: {quality} (R² = {r2:.4f})")

## Вопросы для защиты

1. Какие типы машинного обучения вы знаете?
2. Чем отличается обучение с учителем и без учителя?
3. Чем пакетное обучение отличается от динамического?
4. Чем обучение на основе образцов отличается от обучения на основе модели?
5. Что такое линейная регрессия?
6. Что такое градиентный спуск?
7. Как правильно обрабатывать категориальные признаки?
8. Что такое матрица корреляции?
9. Что показывает метрика RMSE?
10. Что показывает метрика R²?

## Ответы на вопросы 

1. **Какие типы машинного обучения вы знаете?**  
   - Обучение с учителем (supervised)  
   - Обучение без учителя (unsupervised)  
   - Обучение с подкреплением (reinforcement learning)

2. **Чем отличается обучение с учителем и без учителя?**  
   С учителем — есть размеченные данные (признаки + правильный ответ).  
   Без учителя — только признаки, модель сама ищет структуру в данных.

3. **Чем пакетное обучение отличается от динамического?**  
   Пакетное — модель обучается сразу на всех данных.  
   Динамическое (онлайн-обучение) — модель обновляется по мере поступления новых данных.

4. **Чем обучение на основе образцов отличается от обучения на основе модели?**  
   - Образцовое (instance-based): k-NN, предсказывает по похожим примерам  
   - Модельное (model-based): линейная регрессия, строит явную математическую модель

5. **Что такое линейная регрессия?**  
   Метод моделирования зависимости между независимыми переменными (X) и непрерывной целевой переменной (y) с помощью линейного уравнения.

6. **Что такое градиентный спуск?**  
   Алгоритм оптимизации: итеративно корректирует параметры модели, двигаясь в сторону антиградиента функции потерь.

7. **Как правильно обрабатывать категориальные признаки?**  
   - One-Hot Encoding (pd.get_dummies)  
   - Label Encoding / Ordinal Encoding (если есть порядок)  
   - Target Encoding, Frequency Encoding и др.

8. **Что такое матрица корреляции?**  
   Таблица, показывающая коэффициенты корреляции Пирсона между всеми парами числовых признаков. Помогает выявить линейные зависимости.

9. **Что показывает метрика RMSE?**  
   Root Mean Squared Error — среднеквадратичная ошибка в тех же единицах, что и целевая переменная. Чем меньше — тем лучше.

10. **Что показывает метрика R²?**  
   Коэффициент детерминации. Показывает, какую долю дисперсии целевой переменной объясняет модель.  
   R² = 1 — идеальная модель, R² = 0 — как среднее, R² < 0 — хуже среднего.